In [ ]:
%pip install scikit-learn xgboost lime matplotlib seaborn --quiet

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.pipeline import Pipeline
from lime.lime_tabular import LimeTabularExplainer
import matplotlib.pyplot as plt
import seaborn as sns

# Load data
df = pd.read_csv("MachineLearningRating_v3_cleaned.csv")

# Handling Missing Values
df = df.dropna(thresh=len(df.columns) * 0.7)  # Drop rows with more than 30% missing
imputer = SimpleImputer(strategy='mean')
df[['TotalPremium', 'TotalClaims']] = imputer.fit_transform(df[['TotalPremium', 'TotalClaims']])

# Feature Engineering
df['ClaimRatio'] = df['TotalClaims'] / (df['TotalPremium'] + 1)  # Prevent division by zero
df['HighClaim'] = df['ClaimRatio'] > 0.8

# Encoding Categorical Features
cat_cols = df.select_dtypes(include='object').columns
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

# Define target and features
target = 'Retention'  # Replace with your actual target column
X = df.drop(columns=[target])
y = df[target]

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, stratify=y, random_state=42)


In [ ]:
#Modeling
# Initialize models
models = {
    "Logistic Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(n_estimators=100),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss')
}

# Train and evaluate
results = {}

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    results[name] = {
        'model': model,
        'report': report
    }
    print(f"--- {name} ---")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


In [ ]:
#feature importance
importances = {}

for name in ['Random Forest', 'XGBoost']:
    model = results[name]['model']
    feature_imp = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)
    importances[name] = feature_imp

    # Plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x=feature_imp.values[:10], y=feature_imp.index[:10])
    plt.title(f"Top 10 Feature Importances in {name}")
    plt.tight_layout()
    plt.show()


In [ ]:
#LIM interpretability
explainer = LimeTabularExplainer(X_train, feature_names=X.columns, class_names=['No', 'Yes'], verbose=True, mode='classification')

# Choose a single instance to explain
i = 10
exp = explainer.explain_instance(X_test[i], results['XGBoost']['model'].predict_proba)
exp.show_in_notebook(show_table=True)
# Save the model

In [ ]:
#performance comparison
comparison_df = pd.DataFrame({
    model: {
        'Accuracy': results[model]['report']['accuracy'],
        'Precision': results[model]['report']['1']['precision'],
        'Recall': results[model]['report']['1']['recall'],
        'F1-score': results[model]['report']['1']['f1-score']
    }
    for model in results
}).T

print("Model Comparison Report:")
print(comparison_df)

comparison_df.plot(kind='bar', figsize=(12, 6), title='Model Performance Comparison')
plt.ylabel("Score")
plt.xticks(rotation=45)
plt.grid(True)
plt.tight_layout()
plt.show()
